# Doppler Shift

We take $v_{out} = 5$ km/s with $v_{rot} = 0$ km/s. Then we Doppler shift it to reproduce $v_{out} = 5$ km/s with $v_{rot} = 50$ km/s

This is an example:

![example](../../paper/figures/rotation_doppler_outflow.png)

Ly$\alpha$ scattering through an optically thick gas cloud that is undergoing solid-body rotation (i.e. in which the angular speed around the rotation axis is identical for each hydrogen atom) proceeds identical as in a static cloud. In order to compute the spectrum emerging from a rotating cloud, we sum the spectra emerging from all surface elements of the cloud, weighted by their intensity.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

plt.rc('text', usetex=True)
plt.rc('font', size=20)

In [2]:
num_thetas = 9
cos_thetas = np.linspace(0,1,num_thetas+1) 
cos_thetas_plus = np.linspace(-1,0,num_thetas+1) 

thetas = np.sort(np.array([np.arccos(ct) for ct in cos_thetas]))
thetas_plus = np.array([np.arccos(ctp) for ctp in cos_thetas_plus])

In [3]:
### --------------- tau10E5 --------------- ###

vrots = [0,50]
vouts = [25]

all_x_frec = []
all_z_u = []
vrot_labs = []
vout_labs = []

for vout in vouts:
    for vrot in vrots:
        df = pd.read_csv('../../data/tau10E5/vrot'+str(vrot)+'/vout'+str(vout)+'/tau10E5_vrot'+str(vrot)+'_vout'+str(vout)+'_out.ascii', delimiter=' ')
        
        escaped = df['escaped']
        x_frec = df['x_frec']
        z_u = df['z_u']
        
        inds = np.where(escaped == 0)[0]
        
        x_frec_escaped = np.array(x_frec)[inds]
        z_u_escaped = np.array(z_u)[inds]
        
        all_x_frec.append(x_frec_escaped)
        all_z_u.append(z_u_escaped)
              
        vrot_labs.append(vrot)
        vout_labs.append(vout)

/home/mcrg/Software/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
v_th = -12.8

fig = plt.figure(1, (10., 12.))
grid = ImageGrid(fig, 111, # similar to subplot(111)
                nrows_ncols = (2, 1), # creates 2x1 grid of axes
                axes_pad=0.3, # pad between axes in inch.
                aspect=False,)

props = dict(boxstyle='square', facecolor='white')

theta_lower = thetas[-2]
theta_upper = thetas[-1]

theta_plus_lower = thetas_plus[-1]
theta_plus_upper = thetas_plus[-2]

for i in range(2):
    
    filtered_file = open('vrot'+str(vrot_labs[i])+'_vout'+str(vout_labs[i])+'.dat','w') # contains a list of x's within that angle range

    current_z_u = all_z_u[i]
    current_x_frec = all_x_frec[i]

    acos_current_z_u = np.array([np.arccos(czu) for czu in current_z_u])

    angle_indices = np.where( ((acos_current_z_u >= theta_lower) & (acos_current_z_u < theta_upper)) | ((acos_current_z_u >= theta_plus_lower) & (acos_current_z_u < theta_plus_upper)))[0]

    current_x = current_x_frec[angle_indices] #only the ones between those upper and lower angles
    
    for cx in current_x:
        filtered_file.write("%s\n" % cx)
    
    current_x = current_x*v_th

    vrot_lab = r'${0:.0f}$'.format(vrot_labs[i])
    vout_lab = r'${0:.0f}$'.format(vout_labs[i])
    theta_lower_lab = r'${0:.0f}$'.format(int(np.rad2deg(theta_lower)))
    theta_upper_lab = r'${0:.0f}$'.format(int(np.rad2deg(theta_upper)))

    theta_lab = r'$\theta=$ '
    lab_upper = '$v_{out}=$ '+vout_lab+' $\mathrm{km}$ $\mathrm{s^{-1}}$'
    lab = '$v_{rot}=$ '+vrot_lab+' $\mathrm{km}$ $\mathrm{s^{-1}}$'

    grid_i = grid[i]

    n, b = np.histogram(current_x, bins=40)        
    delta_x = b[1]-b[0]
    area = delta_x * sum(n)
    n = n/area
    
    border_x = []
    border_y = []

    for j in range(len(n)-1):
        border_x.append(b[j]+delta_x/2)
        border_x.append(b[j]+delta_x/2)
        border_y.append(n[j])
        border_y.append(n[j+1])
        
    grid_i.bar(b[0:40], n, width=delta_x, color='c', edgecolor=None, alpha = 1)
    grid_i.plot(border_x, border_y, c='k', linewidth=1)

    grid_i.axvline(x=0, ymin=0, ymax=1, c='k', linestyle='--', linewidth=1)
    grid_i.set_xlim(-200,250)
    grid_i.set_ylim(0,0.013)
    grid_i.set_ylabel('$\mathrm{Intensity}$')
    grid_i.set_xlabel('$\mathrm{V}$ ($\mathrm{km}$ $\mathrm{s^{-1}}$)')
    grid_i.text(-180,0.011, lab, fontsize=20, bbox=props)
    if (i==0):
        grid_i.text(-65,0.014, lab_upper, fontsize=20, bbox=props)

    filtered_file.close()

plt.savefig('doppler_shift_original.png', format='png', transparent=False, bbox_inches='tight')
plt.close()